<a href="https://colab.research.google.com/github/surajghuwalewala/CE888_Data_Science_and_Decision_Making/blob/master/Lab_5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv")
df.shape

(24982, 101)

In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(orig!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
val_set, idx = replace(df.values, 0.1)

In [0]:
def latentFactorModelling(data, n_latent_factors = 2, alpha = 0.0001, iterations=300):

  user_ratings = data.values
  # Initialise as random values
  latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
  latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))
  
  
  def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


  def train(user_id, item_id, rating, alpha = alpha):
      
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
      

  def sgd(iterations=iterations):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if (rating != 99.0):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10) == 0:
            print(mse)

  sgd(iterations)

  np.save("latent_factors_{}_iterations.npy".format(iterations), [latent_user_preferences, latent_item_features] )

  return latent_user_preferences, latent_item_features

In [0]:
x = [1,2]
y = [2,3]
np.save("text.npy", [x, y] )

In [7]:
latent_user_preferences, latent_item_features = latentFactorModelling(df, iterations=50)

58.12517708133608
24.55491406393803
20.208689381922422
18.07757880081991


KeyboardInterrupt: ignored

In [8]:
## Train predictions
predictions = latent_user_preferences.dot(latent_item_features.T)

NameError: ignored

In [9]:
## Validation predictions

val_pred = val_set.copy()
for user_id in range(latent_user_preferences.shape[0]):
  for item_id in range(latent_item_features.shape[0]):
    rating = val_set[user_id][item_id]
    if (rating == 99.0):
      prediction_rating = predictions[user_id][item_id]
      val_pred[user_id][item_id] = prediction_rating
    


NameError: ignored

In [0]:
## Test Pred
test_pred = df.values.copy()
for user_id in range(latent_user_preferences.shape[0]):
  for item_id in range(latent_item_features.shape[0]):
    rating = val_set[user_id][item_id]
    if (rating == 99.0):
      prediction_rating = predictions[user_id][item_id]
      test_pred[user_id][item_id] = prediction_rating

In [0]:
values = [zip(df.values[i], predictions[i], val_pred[i], test_pred[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

